# Missing Values
Dealing with missing values with imputation and imputation with extra column

In [2]:
import pandas as pd
house = pd.read_csv("melb_data.csv")

In [3]:
house.describe()

,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,1.358000e+04,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.075684e+06,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,6.393107e+05,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,8.500000e+04,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.500000e+05,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.030000e+05,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,1.330000e+06,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,9.000000e+06,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [4]:
y = house.Price

In [9]:
X = house.drop(['Price'], axis=1)
X_m = X.select_dtypes(exclude=['object']) # remove categorical values

In [10]:
X_m.head()# we can see that there are missing values

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
0,2,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,-37.7996,144.9984,4019.0
1,2,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,-37.8079,144.9934,4019.0
2,3,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,-37.8093,144.9944,4019.0
3,3,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,-37.7969,144.9969,4019.0
4,4,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,-37.8072,144.9941,4019.0


### Method 1. Drop Columns

In [11]:
# import that stuff
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_m, y, random_state=256)

In [13]:
X_train1 = X_train.copy()
X_test1 = X_test.copy()

In [14]:
# identify which columns are missing values
missing = [c for c in X_train1.columns if X_train1[c].isnull().any()]

In [15]:
X_train1 = X_train1.drop(missing, axis=1) # drop the columns
X_test1 = X_test1.drop(missing, axis=1)

In [16]:
# Make this into one function
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def check_model(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mae = mean_absolute_error(predictions, y_test)
    print(mae)

In [17]:
check_model(X_train1, X_test1, y_train, y_test)

183809.3433827232


### Method 2. Imputation with the new column
Have a new column like bed_val_was_missing to indicate that the value was filled in

In [18]:
X_train2 = X_train.copy()
X_test2 = X_test.copy()

In [19]:
# adding new columns
for c in missing: # loop thru each column with missing values
    X_train2[c + '_was_missing'] = X_train2[c].isnull()  # isnull returns a list of boolean
    X_test2[c + '_was_missing'] = X_test2[c].isnull()
    # so you create a new column for each feature with missing values that tells you if the value was missing

In [26]:
# Impute
from sklearn.impute import SimpleImputer
imp = SimpleImputer()

X_train2 = pd.DataFrame(imp.fit_transform(X_train))
X_test2 = pd.DataFrame(imp.transform(X_test))

In [27]:
# Put the column names back
X_train2.columns = X_train.columns
X_test2.columns = X_test.columns

In [22]:
check_model(X_train2, X_test2, y_train, y_test)

181323.49856605186


# Categorical Variables
One hot encoding and ordinal encoding
Make sure to use the X1 we made from before that doesn't have any missing values cause we dropped them

In [34]:
# Drop the columns with missing values so that we can build model and focus on testing stuff with categorical variables instead
missing = [c for c in X.columns if X[c].isnull().any()]
X_C = X.drop(missing, axis=1)

In [35]:
# Split it up
X_trainC, X_testC, y_trainC, y_testC = train_test_split(X_C, y, random_state=256)

In [36]:
s = (X_trainC.dtypes == 'object')
obj_columns = list(s[s].index)

### Method 1. Drop the Categorical Variables
drop the whole column

In [37]:
X_trainC1 = X_trainC.copy()
X_testC1 = X_testC.copy()

In [38]:
#X_trainC = X_trainC.drop(obj_columns, axis=1) # drop using the list we made before, doesn't work cause test and train different lists would need to make 2 lists

X_trainC1 = X_trainC1.select_dtypes(exclude=['object'])
X_testC1 = X_testC1.select_dtypes(exclude=['object'])

In [39]:
check_model(X_trainC1, X_testC1, y_trainC, y_testC)

183998.72946118476


### Method 2. Ordinal Encoding
Encoding the text into numbers, doesn't really work here since there is no order of better home addresses but whatever

In [40]:
X_trainC2 = X_trainC.copy()
X_testC2 = X_testC.copy()

In [41]:
# Fitting an ordinal encoder to a column in the training data creates a corresponding integer-valued label for each unique value that appears in the training data. 
# In the case that the validation data contains values that don't also appear in the training data, the encoder will throw an error, because these values won't have an integer assigned to them

# Categorical columns in the training data
object_cols = [col for col in X_trainC2.columns if X_trainC2[col].dtype == "object"]

# Columns that can be safely ordinal encoded
good_label_cols = [col for col in object_cols if 
                   set(X_testC2[col]).issubset(set(X_trainC2[col]))]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

Categorical columns that will be ordinal encoded: ['Type', 'Method', 'Date', 'Regionname']

Categorical columns that will be dropped from the dataset: ['Address', 'SellerG', 'Suburb']


In [42]:
X_trainC2.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Method', 'SellerG', 'Date',
       'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Landsize', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [43]:
from sklearn.preprocessing import OrdinalEncoder
# Ordinal encoding
encoder = OrdinalEncoder()
X_trainC2[good_label_cols] = encoder.fit_transform(X_trainC2[good_label_cols])
X_testC2[good_label_cols] = encoder.transform(X_testC2[good_label_cols])

# Drop bad columns
X_trainC2 = X_trainC2.drop(bad_label_cols, axis=1)
X_testC2 = X_testC2.drop(bad_label_cols, axis=1)

In [44]:
check_model(X_trainC2, X_testC2, y_train, y_test)

174683.60445360825


### Method 3. One-Hot Encoding
Adding a lot of new columns

In [100]:
X_trainC3 = X_trainC.copy()
X_testC3 = X_testC.copy()

In [101]:
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False) # set handle_unknown to ignore to avoid error

In [102]:
# Categorical columns in the training data
object_cols = [col for col in X_trainC.columns if X_trainC[col].dtype == "object"]

In [103]:
# Columns that will be one-hot encoded
low_cardinality_cols = [col for col in object_cols if X_trainC3[col].nunique() < 10]

# Columns that will be dropped from the dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

Categorical columns that will be one-hot encoded: ['Type', 'Method', 'Regionname']

Categorical columns that will be dropped from the dataset: ['Date', 'Address', 'SellerG', 'Suburb']


In [104]:
# Create NEW datafram with onehot encoded columns from the low cardinality ones only
OH_X_trainC3 = pd.DataFrame(OH_encoder.fit_transform(X_trainC[low_cardinality_cols]))
OH_X_testC3 = pd.DataFrame(OH_encoder.transform(X_testC[low_cardinality_cols]))

In [105]:
# Put the index back
OH_X_trainC3.index = X_trainC.index
OH_X_testC3.index = X_testC.index

In [106]:
# Remove the old categorical columns from the whole dataset
# This is the numerical data
numerical_X_trainC3 = X_trainC.drop(object_cols, axis=1)
numerical_X_testC3 = X_testC.drop(object_cols, axis=1)

In [107]:
X_trainC3N = pd.concat([numerical_X_trainC3, OH_X_trainC3], axis=1) # join the numerical data together with the onehot encoded dataframe
X_testC3N = pd.concat([numerical_X_testC3, OH_X_testC3], axis=1)

In [108]:
check_model(X_trainC3N, X_testC3N, y_train, y_test)

174218.45805161653


In [113]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=500) # higher since early stopping
model.fit(X_trainC3N, y_train, early_stopping_rounds=5, eval_set=[(X_testC3N, y_test)], verbose=False) # can also set to true for text progress

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [115]:
pred = model.predict(X_testC3N)
mae = mean_absolute_error(pred, y_test)

In [116]:
mae

171101.31754418262